In [ ]:
import json
import minsearch

In [ ]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [ ]:
documents = []
for course_dict in docs_raw:
    for document in course_dict['documents']:
        document['course'] = course_dict['course']
        documents.append(document)


In [ ]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [ ]:
index.fit(documents)

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "OPENAI_API_KEY= environment variable is not set.")

In [ ]:
client = OpenAI()

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}]
    )

    return response.choices[0].message.content

In [ ]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )

    return results

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
    Use only the facts from the CONTEXT content.
    If the CONTEXT doesn't contain the ANSWER output NONE.

    QUESTION: {question}

    CONTEXT: {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(
        question=query,
        context=context
    ).strip()

    return prompt

In [ ]:
build_prompt(query, search(query))

In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [ ]:
print(rag("how to install postgres?"))

In [ ]:
documents[0]

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
es_client.info()

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(
    index=index_name,
    body=index_settings
)

In [ ]:
documents[0]

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for document in tqdm(documents):
    es_client.index(
        index=index_name,
        document=document
    )

In [ ]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
query = 'I just discovered the course. Can I still join?'

In [ ]:
elastic_search(query)

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [ ]:
rag(query)